In [1]:
import os
import pickle
import re
from pathlib import Path
import pandas as pd


In [2]:
rel_dwell = pd.read_excel("S03_af_feature_relevance.xlsx", sheet_name="af_dwell")
rel_weld = pd.read_excel("S03_af_feature_relevance.xlsx", sheet_name="af_weld")

In [3]:
rel_dwell

,feature,type,p_value,relevant,y
0,"Fz__change_quantiles__f_agg_""var""__isabs_True_...",real,0.001835,False,Center_L7
1,"Fz__change_quantiles__f_agg_""mean""__isabs_True...",real,0.004020,False,Center_L7
2,"Fz__fft_coefficient__attr_""real""__coeff_35",real,0.004418,False,Center_L7
3,"Fz__change_quantiles__f_agg_""var""__isabs_False...",real,0.004629,False,Center_L7
4,"Fz__fft_coefficient__attr_""real""__coeff_39",real,0.006105,False,Center_L7
...,...,...,...,...,...
1559,Fz__ratio_beyond_r_sigma__r_6,constant,NaN,False,Center_mean
1560,Fz__ratio_beyond_r_sigma__r_7,constant,NaN,False,Center_mean
1561,Fz__ratio_beyond_r_sigma__r_10,constant,NaN,False,Center_mean
1562,Fz__count_above__t_0,constant,NaN,False,Center_mean


In [4]:
rel_weld

,feature,type,p_value,relevant,y
0,"Fz__change_quantiles__f_agg_""mean""__isabs_Fals...",real,0.000152,False,Center_L7
1,"Fz__change_quantiles__f_agg_""mean""__isabs_Fals...",real,0.000433,False,Center_L7
2,"Fz__change_quantiles__f_agg_""mean""__isabs_Fals...",real,0.001153,False,Center_L7
3,"Fz__change_quantiles__f_agg_""mean""__isabs_Fals...",real,0.001153,False,Center_L7
4,"Fz__change_quantiles__f_agg_""mean""__isabs_Fals...",real,0.001349,False,Center_L7
...,...,...,...,...,...
1559,Fz__ratio_beyond_r_sigma__r_6,constant,NaN,False,Center_mean
1560,Fz__ratio_beyond_r_sigma__r_7,constant,NaN,False,Center_mean
1561,Fz__ratio_beyond_r_sigma__r_10,constant,NaN,False,Center_mean
1562,Fz__count_above__t_0,constant,NaN,False,Center_mean


In [5]:
display(rel_dwell[rel_dwell["relevant"]])

,feature,type,p_value,relevant,y


In [6]:
display(rel_weld[rel_weld["relevant"]])

,feature,type,p_value,relevant,y


In [7]:
def apply_select(df, num=5):
    # print(df.name)
    filt_p = (df["p_value"] <= 0.05) | df["relevant"]
    df_filt = df[filt_p]
    df_filt = df_filt.sort_values(by="p_value", ascending=True).reset_index(drop=True).head(num)
    return df_filt

# data = rel_dwell
# dfg = data.groupby("y")
# group_names = list(dfg.groups.keys())
# name = group_names[0]
# df = dfg.get_group(name)
# filt_p = (df["p_value"] <= 0.05) | df["relevant"]
# df_filt = df[filt_p]
# display(df_filt)

In [8]:
# Select features for dwell 
dfg = rel_dwell.groupby("y")
_select = dfg.apply(lambda df: apply_select(df, num=10), include_groups=False)
_select = _select.reset_index().drop(columns=["level_1"])
print(f"Selected features (1st round) for dwell: {_select.shape[0]}")
filt_dup = _select.duplicated(subset=["feature"])
print(f"Duplicated features: {_select[filt_dup].shape[0]}")
select = _select[~filt_dup]
print(f"Selected features: {select.shape[0]}")
display(_select[~filt_dup])
# Save selected features
select_dwell = select.copy()


Selected features (1st round) for dwell: 20
Duplicated features: 10
Selected features: 10


,y,feature,type,p_value,relevant
0,Center_L7,"Fz__change_quantiles__f_agg_""var""__isabs_True_...",real,0.001835,False
1,Center_L7,"Fz__change_quantiles__f_agg_""mean""__isabs_True...",real,0.004020,False
2,Center_L7,"Fz__fft_coefficient__attr_""real""__coeff_35",real,0.004418,False
3,Center_L7,"Fz__change_quantiles__f_agg_""var""__isabs_False...",real,0.004629,False
4,Center_L7,"Fz__fft_coefficient__attr_""real""__coeff_39",real,0.006105,False
5,Center_L7,"Fz__fft_coefficient__attr_""real""__coeff_69",real,0.006388,False
6,Center_L7,"Fz__fft_coefficient__attr_""real""__coeff_64",real,0.008352,False
7,Center_L7,"Fz__change_quantiles__f_agg_""var""__isabs_False...",real,0.009118,False
8,Center_L7,"Fz__fft_coefficient__attr_""real""__coeff_82",real,0.012844,False
9,Center_L7,"Fz__cwt_coefficients__coeff_1__w_5__widths_(2,...",real,0.015807,False


In [9]:
# Select features for weld
dfg = rel_weld.groupby("y")
_select = dfg.apply(lambda df: apply_select(df, num=10), include_groups=False)
_select = _select.reset_index().drop(columns=["level_1"])
print(f"Selected features (1st round) for weld: {_select.shape[0]}")
filt_dup = _select.duplicated(subset=["feature"])
print(f"Duplicated features: {_select[filt_dup].shape[0]}")
select = _select[~filt_dup]
print(f"Selected features: {select.shape[0]}")
display(_select[~filt_dup])
# Save selected features
select_weld = select.copy()

Selected features (1st round) for weld: 20
Duplicated features: 10
Selected features: 10


,y,feature,type,p_value,relevant
0,Center_L7,"Fz__change_quantiles__f_agg_""mean""__isabs_Fals...",real,0.000152,False
1,Center_L7,"Fz__change_quantiles__f_agg_""mean""__isabs_Fals...",real,0.000433,False
2,Center_L7,"Fz__change_quantiles__f_agg_""mean""__isabs_Fals...",real,0.001153,False
3,Center_L7,"Fz__change_quantiles__f_agg_""mean""__isabs_Fals...",real,0.001153,False
4,Center_L7,"Fz__change_quantiles__f_agg_""mean""__isabs_Fals...",real,0.001349,False
5,Center_L7,"Fz__change_quantiles__f_agg_""mean""__isabs_Fals...",real,0.001835,False
6,Center_L7,Fz__lempel_ziv_complexity__bins_2,real,0.004314,False
7,Center_L7,"Fz__fft_coefficient__attr_""imag""__coeff_3",real,0.004418,False
8,Center_L7,"Fz__fft_coefficient__attr_""abs""__coeff_0",real,0.004850,False
9,Center_L7,"Fz__fft_coefficient__attr_""real""__coeff_0",real,0.004850,False


In [10]:
with pd.ExcelWriter("S04_af_feature_selection.xlsx", engine="openpyxl") as writer:
    select_dwell.to_excel(writer, sheet_name="af_dwell", index=False)
    select_weld.to_excel(writer, sheet_name="af_weld", index=False)